[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Hawksight-AI/semantica/blob/main/cookbook/use_cases/supply_chain/02_Supply_Chain_Risk_Management.ipynb)

# Supply Chain Risk Management with Semantica

## Overview

This notebook demonstrates using **Semantica as the core framework** to transform supply chains into dynamic, interconnected networks for real-time visualization, analysis, and proactive risk management.


**Documentation**: [API Reference](https://semantica.readthedocs.io/use-cases/)

### Why Semantica?

Semantica provides comprehensive graph technology capabilities essential for supply chain risk management:

- **Graph Modeling**: Semantica's KG modules naturally model complex supply chain relationships and dependencies
- **Real-Time Analysis**: Semantica's GraphAnalyzer enables real-time network analysis and risk assessment
- **Cascade Effect Analysis**: Semantica's ConnectivityAnalyzer identifies how disruptions cascade through networks
- **Graph Algorithms**: Semantica's graph analytics provide betweenness centrality for alternative sourcing identification
- **Risk Clustering**: Semantica's community detection identifies supplier risk clusters
- **Visualization**: Semantica's visualization modules provide real-time supply chain network visualization

### Key Features

- Graph modeling of complex supply chain relationships and dependencies using Semantica
- Real-time visualization of supply chain networks
- Risk analysis for tariffs, extreme weather, component shortages
- Cascade effect analysis through network
- Alternative sourcing identification using Semantica's graph algorithms (betweenness centrality)
- Product redesign impact modeling
- Supplier risk clustering

### Semantica Modules Used (15+)

- **Ingest**: FileIngestor, WebIngestor, FeedIngestor, StreamIngestor, DBIngestor, RepoIngestor, EmailIngestor, MCPIngestor (supply chain data from multiple sources)
- **Parse**: StructuredDataParser, JSONParser, CSVParser (BOM data, supplier data, tariff data)
- **Normalize**: TextNormalizer, DataNormalizer (for data standardization)
- **Semantic Extract**: RelationExtractor, TripletExtractor (supplier relationships, dependencies, risk factors)
- **KG**: GraphBuilder, GraphAnalyzer, ConnectivityAnalyzer (supply chain graph construction)
- **Graph Analytics**: Use Semantica's GraphAnalyzer for community detection, centrality measures (PageRank, Betweenness, Closeness)
- **Embeddings**: EmbeddingGenerator (for supplier similarity, risk clustering)
- **Vector Store**: VectorStore, HybridSearch (for supplier search and risk analysis)
- **Reasoning**: Reasoner (Legacy), RuleManager (for risk propagation rules, tariff impact analysis)
- **Seed**: SeedDataManager (for loading supplier master data)
- **Visualization**: KGVisualizer, AnalyticsVisualizer (network visualization, risk heatmaps, supply chain dashboards)
- **Export**: JSONExporter, CSVExporter, ReportGenerator (for risk reports)
- **Pipeline**: PipelineBuilder, ExecutionEngine (for end-to-end supply chain analysis pipeline)

### Pipeline Overview

**Supply Chain Data → Parse → Extract Relationships → Build Supply Chain Graph → Analyze Risks → Identify Alternatives → Visualize → Generate Reports**

## Installation

Install Semantica from PyPI:

```bash
pip install semantica
# Or with all optional dependencies:
pip install semantica[all]
```

---

## Step 1: Setup and Import Semantica Modules


In [ ]:
!pip install semantica


In [ ]:
# Import all Semantica modules - using Semantica as the core framework
from semantica.ingest import FileIngestor, DBIngestor, WebIngestor, StreamIngestor
from semantica.parse import StructuredDataParser, JSONParser, CSVParser
from semantica.normalize import TextNormalizer, DataNormalizer
from semantica.semantic_extract import RelationExtractor, TripletExtractor
from semantica.kg import GraphBuilder, GraphAnalyzer, ConnectivityAnalyzer
from semantica.embeddings import EmbeddingGenerator
from semantica.vector_store import VectorStore, HybridSearch
# # from semantica.reasoning import InferenceEngine, RuleManager
from semantica.seed import SeedDataManager
from semantica.visualization import KGVisualizer, AnalyticsVisualizer
from semantica.export import JSONExporter, CSVExporter, ReportGenerator
from semantica.pipeline import PipelineBuilder, ExecutionEngine

import tempfile
import os
import json
from datetime import datetime, timedelta
import numpy as np


## Step 2: Ingest Supply Chain Data Using Semantica

Using Semantica's ingest modules to load supply chain data including BOM (Bill of Materials), supplier information, and tariff data.


In [ ]:
# Initialize Semantica ingestors
file_ingestor = FileIngestor()
db_ingestor = DBIngestor()
web_ingestor = WebIngestor()
stream_ingestor = StreamIngestor()
seed_loader = SeedDataManager()

# Create temporary directory for sample data
temp_dir = tempfile.mkdtemp()

# Sample BOM (Bill of Materials) data
bom_data = {
    "product_id": "PROD-001",
    "product_name": "Electronic Device",
    "components": [
        {"component_id": "COMP-001", "name": "Processor", "supplier_id": "SUP-001", "quantity": 1},
        {"component_id": "COMP-002", "name": "Memory", "supplier_id": "SUP-002", "quantity": 2},
        {"component_id": "COMP-003", "name": "Display", "supplier_id": "SUP-003", "quantity": 1},
        {"component_id": "COMP-004", "name": "Battery", "supplier_id": "SUP-001", "quantity": 1}
    ]
}

# Sample supplier data
supplier_data = {
    "suppliers": [
        {
            "supplier_id": "SUP-001",
            "name": "Global Electronics Inc",
            "location": "China",
            "risk_factors": ["tariff_impact", "weather_risk"],
            "alternatives": ["SUP-005", "SUP-006"]
        },
        {
            "supplier_id": "SUP-002",
            "name": "Memory Solutions Ltd",
            "location": "South Korea",
            "risk_factors": ["tariff_impact"],
            "alternatives": ["SUP-007"]
        },
        {
            "supplier_id": "SUP-003",
            "name": "Display Tech Corp",
            "location": "Taiwan",
            "risk_factors": ["weather_risk"],
            "alternatives": ["SUP-008"]
        }
    ]
}

# Sample tariff data
tariff_data = {
    "tariffs": [
        {
            "tariff_id": "TAR-001",
            "country": "China",
            "category": "Electronics",
            "rate": 0.25,
            "effective_date": "2025-01-01",
            "impacted_suppliers": ["SUP-001"]
        }
    ]
}

# Save sample data
bom_file = os.path.join(temp_dir, "bom.json")
supplier_file = os.path.join(temp_dir, "suppliers.json")
tariff_file = os.path.join(temp_dir, "tariffs.json")

with open(bom_file, 'w') as f:
    json.dump(bom_data, f, indent=2)

with open(supplier_file, 'w') as f:
    json.dump(supplier_data, f, indent=2)

with open(tariff_file, 'w') as f:
    json.dump(tariff_data, f, indent=2)

# Ingest using Semantica FileIngestor
bom_file_obj = file_ingestor.ingest_file(bom_file, read_content=True)
supplier_file_obj = file_ingestor.ingest_file(supplier_file, read_content=True)
tariff_file_obj = file_ingestor.ingest_file(tariff_file, read_content=True)

# Load supplier master data using Semantica SeedDataManager
supplier_seed_data = seed_loader.load_from_json(supplier_file)


In [ ]:
# Initialize Semantica parsers and normalizers
structured_parser = StructuredDataParser()
json_parser = JSONParser()
csv_parser = CSVParser()
text_normalizer = TextNormalizer()
data_normalizer = DataNormalizer()

# Parse BOM data using Semantica
parsed_bom = structured_parser.parse_data(bom_file, data_format="json")
bom_data_parsed = parsed_bom.get("data") if isinstance(parsed_bom, dict) else parsed_bom

# Parse supplier data using Semantica
parsed_suppliers = structured_parser.parse_data(supplier_file, data_format="json")
supplier_data_parsed = parsed_suppliers.get("data") if isinstance(parsed_suppliers, dict) else parsed_suppliers

# Parse tariff data using Semantica
parsed_tariffs = structured_parser.parse_data(tariff_file, data_format="json")
tariff_data_parsed = parsed_tariffs.get("data") if isinstance(parsed_tariffs, dict) else parsed_tariffs

# Normalize supplier names using Semantica
if isinstance(supplier_data_parsed, dict):
    for supplier in supplier_data_parsed.get('suppliers', []):
        supplier['normalized_name'] = text_normalizer.normalize_text(supplier.get('name', ''))


## Step 4: Extract Supply Chain Relationships Using Semantica

Using Semantica's semantic extraction modules to extract supplier relationships, dependencies, and risk factors.


In [ ]:
# Initialize Semantica extractors
relation_extractor = RelationExtractor()
triplet_extractor = TripletExtractor()

# Build supply chain entities and relationships
supply_chain_entities = []
supply_chain_relationships = []

# Add product entity
if isinstance(bom_data_parsed, dict):
    supply_chain_entities.append({
        "id": bom_data_parsed.get('product_id', ''),
        "type": "Product",
        "name": bom_data_parsed.get('product_name', ''),
        "properties": {}
    })

# Add component entities and relationships
if isinstance(bom_data_parsed, dict):
    for component in bom_data_parsed.get('components', []):
        supply_chain_entities.append({
            "id": component.get('component_id', ''),
            "type": "Component",
            "name": component.get('name', ''),
            "properties": {
                "quantity": component.get('quantity', 0)
            }
        })
        
        # Product-Component relationship
        supply_chain_relationships.append({
            "source": bom_data_parsed.get('product_id', ''),
            "target": component.get('component_id', ''),
            "type": "contains",
            "properties": {
                "quantity": component.get('quantity', 0)
            }
        })
        
        # Component-Supplier relationship
        supply_chain_relationships.append({
            "source": component.get('component_id', ''),
            "target": component.get('supplier_id', ''),
            "type": "supplied_by",
            "properties": {}
        })

# Add supplier entities
if isinstance(supplier_data_parsed, dict):
    for supplier in supplier_data_parsed.get('suppliers', []):
        supply_chain_entities.append({
            "id": supplier.get('supplier_id', ''),
            "type": "Supplier",
            "name": supplier.get('name', ''),
            "properties": {
                "location": supplier.get('location', ''),
                "risk_factors": supplier.get('risk_factors', [])
            }
        })
        
        # Alternative supplier relationships
        for alt_supplier in supplier.get('alternatives', []):
            supply_chain_relationships.append({
                "source": supplier.get('supplier_id', ''),
                "target": alt_supplier,
                "type": "alternative_to",
                "properties": {}
            })

# Add tariff entities and relationships
if isinstance(tariff_data_parsed, dict):
    for tariff in tariff_data_parsed.get('tariffs', []):
        supply_chain_entities.append({
            "id": tariff.get('tariff_id', ''),
            "type": "Tariff",
            "name": f"Tariff {tariff.get('country', '')}",
            "properties": {
                "rate": tariff.get('rate', 0),
                "effective_date": tariff.get('effective_date', '')
            }
        })
        
        # Tariff-Supplier relationship
        for supplier_id in tariff.get('impacted_suppliers', []):
            supply_chain_relationships.append({
                "source": tariff.get('tariff_id', ''),
                "target": supplier_id,
                "type": "impacts",
                "properties": {
                    "impact_type": "tariff"
                }
            })

print(f"  - Products: {len([e for e in supply_chain_entities if e.get('type') == 'Product'])}")
print(f"  - Components: {len([e for e in supply_chain_entities if e.get('type') == 'Component'])}")
print(f"  - Suppliers: {len([e for e in supply_chain_entities if e.get('type') == 'Supplier'])}")
print(f"  - Tariffs: {len([e for e in supply_chain_entities if e.get('type') == 'Tariff'])}")


## Step 5: Build Supply Chain Knowledge Graph Using Semantica

Using Semantica's KG modules to build a supply chain knowledge graph.


In [ ]:
# Initialize Semantica KG builders and analyzers
graph_builder = GraphBuilder()
graph_analyzer = GraphAnalyzer()
connectivity_analyzer = ConnectivityAnalyzer()

# Build supply chain knowledge graph using Semantica
supply_chain_kg = graph_builder.build(supply_chain_entities, supply_chain_relationships)

# Analyze the graph using Semantica
kg_metrics = graph_analyzer.compute_metrics(supply_chain_kg)
connectivity = connectivity_analyzer.analyze_connectivity(supply_chain_kg)

print(f"  - Entities: {len(supply_chain_kg.get('entities', []))}")
print(f"  - Relationships: {len(supply_chain_kg.get('relationships', []))}")
print(f"  - Graph density: {kg_metrics.get('density', 0):.4f}")
print(f"  - Connected components: {connectivity.get('num_components', 0)}")


## Step 6: Analyze Supply Chain Risks Using Semantica Graph Analytics

Using Semantica's GraphAnalyzer to perform risk analysis including community detection, centrality measures, and cascade effect analysis.


In [ ]:
# Perform graph analytics using Semantica

# 1. Community detection for supplier risk clustering
communities = graph_analyzer.detect_communities(supply_chain_kg, method="louvain")

# 2. Centrality measures for alternative sourcing identification
pagerank_centrality = graph_analyzer.compute_centrality(supply_chain_kg, method="pagerank")
betweenness_centrality = graph_analyzer.compute_centrality(supply_chain_kg, method="betweenness")
closeness_centrality = graph_analyzer.compute_centrality(supply_chain_kg, method="closeness")

# 3. Identify critical suppliers (high betweenness = alternative sourcing options)
critical_suppliers = sorted(
    [(node, score) for node, score in betweenness_centrality.items() if node.startswith('SUP-')],
    key=lambda x: x[1],
    reverse=True
)[:5]

# 4. Cascade effect analysis - identify suppliers at risk
at_risk_suppliers = []
for entity in supply_chain_entities:
    if entity.get('type') == 'Supplier':
        risk_factors = entity.get('properties', {}).get('risk_factors', [])
        if risk_factors:
            at_risk_suppliers.append({
                "supplier_id": entity.get('id'),
                "supplier_name": entity.get('name'),
                "risk_factors": risk_factors,
                "betweenness_centrality": betweenness_centrality.get(entity.get('id'), 0)
            })

print(f"  - Communities detected: {communities.get('num_communities', 0)}")
print(f"  - Critical suppliers (high betweenness): {len(critical_suppliers)}")
print(f"  - At-risk suppliers: {len(at_risk_suppliers)}")
print(f"\nTop Critical Suppliers (Alternative Sourcing Options):")
for supplier_id, score in critical_suppliers:
    supplier_name = next((e.get('name', '') for e in supply_chain_entities if e.get('id') == supplier_id), 'Unknown')
    print(f"  - {supplier_name} ({supplier_id}): {score:.4f}")


## Step 7: Implement Risk Propagation Rules Using Semantica Reasoning

Using Semantica's reasoning modules to implement risk propagation rules for tariff impacts and cascade effects.


In [ ]:
# Initialize Semantica reasoning modules
# # inference_engine = InferenceEngine()
rule_manager = RuleManager()

# Define risk propagation rules using Semantica
risk_rules = [
    {
        "rule_id": "tariff_impact_rule",
        "condition": "IF supplier has tariff_impact AND supplier supplies component THEN component has cost_increase",
        "action": "propagate_cost_increase"
    },
    {
        "rule_id": "cascade_effect_rule",
        "condition": "IF supplier has risk_factor AND supplier is critical THEN product has supply_risk",
        "action": "flag_supply_risk"
    },
    {
        "rule_id": "alternative_sourcing_rule",
        "condition": "IF supplier has high_betweenness_centrality THEN supplier is alternative_sourcing_option",
        "action": "identify_alternative"
    }
]

# Add rules using Semantica
for rule in risk_rules:
    rule_manager.add_rule(rule)

# # # Apply risk propagation using Semantica InferenceEngine
risk_analysis_results = inference_engine.infer(
    knowledge_graph=supply_chain_kg,
    rules=risk_rules,
    facts=at_risk_suppliers
)

print(f"  - Rules defined: {len(risk_rules)}")
print(f"  - Risk analysis results: {len(risk_analysis_results) if isinstance(risk_analysis_results, list) else 1}")
print(f"  - Cascade effects identified: Enabled")


In [ ]:
# Initialize Semantica visualizers
kg_visualizer = KGVisualizer(layout="force", color_scheme="vibrant")
analytics_visualizer = AnalyticsVisualizer()

# Visualize supply chain network using Semantica
network_fig = kg_visualizer.visualize_network(
    supply_chain_kg,
    output="interactive"
)

# Visualize communities (supplier risk clusters) using Semantica
communities_data = {
    "graph": supply_chain_kg,
    "communities": communities
}
communities_fig = kg_visualizer.visualize_communities(
    supply_chain_kg,
    communities,
    output="interactive"
)

# Visualize centrality rankings using Semantica
centrality_fig = analytics_visualizer.visualize_centrality_rankings(
    betweenness_centrality,
    centrality_type="betweenness",
    top_n=10,
    output="interactive"
)

# Create risk dashboard data
risk_dashboard_data = {
    "overall_score": 0.75,
    "tariff_risk_score": 0.80,
    "weather_risk_score": 0.60,
    "supply_risk_score": 0.70
}


## Step 9: Generate Risk Reports Using Semantica

Using Semantica's export modules to generate comprehensive risk reports.


In [ ]:
# Initialize Semantica exporters
json_exporter = JSONExporter()
csv_exporter = CSVExporter()
report_generator = ReportGenerator()

# Export supply chain graph as JSON using Semantica
kg_json_file = os.path.join(temp_dir, "supply_chain_kg.json")
json_exporter.export(supply_chain_kg, kg_json_file)

# Export at-risk suppliers as CSV using Semantica
at_risk_suppliers_csv = os.path.join(temp_dir, "at_risk_suppliers.csv")
csv_exporter.export(at_risk_suppliers, at_risk_suppliers_csv)

# Generate comprehensive risk report using Semantica
risk_report_data = {
    "title": "Supply Chain Risk Management Report",
    "knowledge_graph_metrics": kg_metrics,
    "at_risk_suppliers": at_risk_suppliers,
    "critical_suppliers": critical_suppliers,
    "risk_analysis": risk_analysis_results,
    "risk_dashboard": risk_dashboard_data
}
risk_report_file = os.path.join(temp_dir, "supply_chain_risk_report.html")
report_generator.generate_report(risk_report_data, risk_report_file, format="html")

print(f"  - Knowledge graph JSON: {kg_json_file}")
print(f"  - At-risk suppliers CSV: {at_risk_suppliers_csv}")
print(f"  - Risk report HTML: {risk_report_file}")


## Step 10: Complete Pipeline Orchestration Using Semantica

Using Semantica's pipeline module to orchestrate the complete supply chain risk analysis pipeline.


In [ ]:
# Build complete pipeline using Semantica PipelineBuilder
pipeline_builder = PipelineBuilder()

def ingest_handler(data, **config):
    source_dir = config.get("source", temp_dir)
    files = []
    try:
        files = [os.path.join(source_dir, f) for f in os.listdir(source_dir)]
    except Exception:
        pass
    return {**data, "files": files}
def parse_handler(data, **config):
    records = []
    files = data.get("files", [])
    for fp in files:
        try:
            with open(fp, "r", encoding="utf-8") as f:
                records.append(json.load(f))
        except Exception:
            pass
    return {**data, "records": records}
def normalize_handler(data, **config):
    records = data.get("records", [])
    normalized = records
    return {**data, "normalized": normalized}
def extract_handler(data, **config):
    normalized = data.get("normalized", [])
    entities = data.get("entities") or []
    relationships = data.get("relationships") or []
    return {**data, "entities": entities, "relationships": relationships}
def build_kg_handler(data, **config):
    ents = data.get("entities", [])
    rels = data.get("relationships", [])
    kg = graph_builder.build({"entities": ents, "relationships": rels})
    return {**data, "supply_chain_kg": kg}
def analyze_risks_handler(data, **config):
    kg = data.get("supply_chain_kg")
    risks = graph_analyzer.compute_metrics(kg) if kg is not None else {}
    return {**data, "risk_analysis": risks}
def propagate_risks_handler(data, **config):
    kg = data.get("supply_chain_kg")
    propagation = reasoning_engine.infer(knowledge_graph=kg, rules=[]) if kg is not None else {}
    return {**data, "risk_propagation": propagation}
def visualize_handler(data, **config):
    kg = data.get("supply_chain_kg")
    fig = KGVisualizer().visualize_network(kg) if kg is not None else None
    return {**data, "visualization": fig}
def report_handler(data, **config):
    report = ReportGenerator().generate({"risks": data.get("risk_analysis"), "propagation": data.get("risk_propagation")})
    return {**data, "report": report}
supply_chain_pipeline = (
    pipeline_builder
    .add_step("ingest", "ingest", handler=ingest_handler, source=temp_dir)
    .add_step("parse", "parse", dependencies=["ingest"], handler=parse_handler, formats=["json"])
    .add_step("normalize", "normalize", dependencies=["parse"], handler=normalize_handler)
    .add_step("extract", "extract", dependencies=["normalize"], handler=extract_handler)
    .add_step("build_kg", "build_kg", dependencies=["extract"], handler=build_kg_handler)
    .add_step("analyze_risks", "analyze_graph", dependencies=["build_kg"], handler=analyze_risks_handler)
    .add_step("propagate_risks", "reasoning", dependencies=["build_kg"], handler=propagate_risks_handler)
    .add_step("visualize", "visualize", dependencies=["analyze_risks"], handler=visualize_handler)
    .add_step("generate_report", "report", dependencies=["visualize"], handler=report_handler)
    .build()
)

execution_engine = ExecutionEngine()
input_data = {"entities": supply_chain_entities if 'supply_chain_entities' in globals() else [], "relationships": supply_chain_relationships if 'supply_chain_relationships' in globals() else []}
pipeline_result = execution_engine.execute_pipeline(supply_chain_pipeline, data=input_data)

print(f"  - Pipeline steps: {len(supply_chain_pipeline.steps)}")
print(f"  - Execution status: {pipeline_result.success if hasattr(pipeline_result, 'success') else 'Completed'}")


## Conclusion and Best Practices

### Key Takeaways

1. **Semantica as Core Framework**: This notebook demonstrated using Semantica as the exclusive framework for supply chain risk management
2. **Graph Modeling**: Semantica's KG modules naturally model complex supply chain relationships and dependencies
3. **Risk Analysis**: Semantica's GraphAnalyzer provides powerful algorithms for risk identification and alternative sourcing
4. **Cascade Effects**: Semantica's ConnectivityAnalyzer identifies how disruptions cascade through supply networks
5. **Real-Time Visualization**: Semantica's visualization modules provide real-time supply chain network visualization
6. **Risk Propagation**: Semantica's Reasoning modules enable rule-based risk propagation analysis

### Semantica-Specific Performance Considerations

- **Graph Analytics**: Use Semantica's GraphAnalyzer for efficient centrality and community detection on large supply chains
- **Batch Processing**: Leverage Semantica's batch processing for large-scale supplier data ingestion
- **Caching**: Utilize Semantica's caching for frequently accessed supplier relationships
- **Parallel Execution**: Use Semantica's ExecutionEngine for parallel risk analysis

### Deployment Recommendations Using Semantica

1. **Production Setup**:
   - Use Semantica's configuration management for supply chain data sources
   - Leverage Semantica's Pipeline module for automated risk monitoring
   - Use Semantica's StreamIngestor for real-time supply chain data updates

2. **Scalability**:
   - Use Semantica's batch processing for large supplier networks
   - Leverage Semantica's graph analytics optimizations
   - Utilize Semantica's parallel execution for concurrent risk analysis

3. **Real-Time Monitoring**:
   - Use Semantica's StreamIngestor for real-time tariff and weather updates
   - Leverage Semantica's visualization modules for live dashboards
   - Utilize Semantica's reasoning modules for automated risk alerts

### How Semantica's Architecture Benefits Supply Chain Risk Management

- **Natural Graph Modeling**: Semantica's graph structure naturally represents supply chain relationships
- **Comprehensive Analytics**: Semantica's GraphAnalyzer provides all necessary algorithms (centrality, communities, connectivity)
- **Extensibility**: Semantica's registry system enables custom risk analysis methods
- **Integration**: Semantica's unified framework simplifies integration with existing supply chain systems
- **Performance**: Semantica's optimized algorithms handle large-scale supply chain networks efficiently
